### Závěrečný projekt - modul Python

V tomto projektu budete analyzovat data ze stránky `www.sreality.cz`. Budete tedy získávat aktuální nabídku bytů ve Zlíně.
Projekt se dělí na 2 části:
1. WebScraping - pomocí knihovny `BeautifulSoup` získejte všechny aktuální inzerce bytů v daném městě. 
- Získáte **tyto** informace: `URL odkaz bytu, rozměry bytu, kompozice bytu, cena bytu, lokace bytu (Ulice+město)`
2. Datová analýza - ETL proces
- V první fázi prověďte processing dat tak, aby obsahovala příslušné datové typy, nenulové hodnoty atp. Ulice i město bude v samostatných sloupcích.
- Zprocesovaná a vyčištěná data vyexportujte do `.csv` souboru s názvem `{vašeJméno}_zdrojová_data.csv`
3. Datová analýza - analýza dat
- Třetí část začněte tak, že si do dataframe nahrajete vámi vyexportovaný `.csv` soubor s vyscrapovanými daty.
- Ve druhé fázi proveďte analýzu dat a odpovězte na tyto dotazy:
- `Jaká je průměrná cena bytů?`
- `Jaká je průměrná cena bytů pro každou kompozici (1+1, 2+1, atp.)? Zobrazte i na grafu.`
- `Jaká je průměrná velikost bytu pro každou kompozici?`
- `Existuje ulice, kde je vyšší koncentrace dražších bytů?`
- `Jaký typ kompozice je v daném městě nejčastěji inzerovan? Který to je? Proč myslíte, že tomu tak je?`
- `Existují zde inzerce bytů, které stojí více, než 20.000,-? Pokud ano, jsou v této cenové hladině inzerovány i maximálně dvoupokojové byty (2+1/2+kk)?`
- `Zjištěte minimální a maximální inzerce pro každou kompozici bytu. Která kompozice má největší rozptyl mezi minimální a maximální inzerovanou cenou?`


Dotazy zodpovězte ve vašem pracovním notebooku a výsledný soubor nahrajte na GitHub.


### WebScraping

In [ ]:
# 1. WebScraping - pomocí knihovny `BeautifulSoup` získejte všechny aktuální inzerce bytů v daném městě. 
#- Získáte **tyto** informace: `URL odkaz bytu, rozměry bytu, kompozice bytu, cena bytu, lokace bytu (Ulice+město)`

In [1]:
from bs4 import BeautifulSoup
import requests


strana = 1
list_apartments = []

while True:
    mesto = 'zlin'
    data = requests.get(f'https://www.sreality.cz/hledani/pronajem/byty/{mesto}?strana={strana}').text
    soup = BeautifulSoup(data, 'html.parser')
    apartments = soup.findAll('li', class_='MuiGrid-root MuiGrid-item css-l1328q')


    for apartment in apartments:
        je_inzerat = apartment.find('div', 'MuiBox-root css-n6y9a7')
        if je_inzerat is None:
            continue
        print(apartment)
        #find URL
        url = apartment.find('a', class_='MuiTypography-root MuiTypography-inherit MuiLink-root MuiLink-underlineAlways css-1c7vz4z', attrs={'href'})['href']
        url_full = 'https://www.sreality.cz/' + url
        #find dimension
        dimensions = apartment.find('p', class_='MuiTypography-root MuiTypography-body1 css-13ztabn').text.replace('Pronájem bytu ', '').replace('\xa0m²', '').split(' ')[1]
        print(dimensions)
        #find disposition
        disposition = apartment.find('p', class_='MuiTypography-root MuiTypography-body1 css-13ztabn').text.replace('Pronájem bytu ', '').split(' ')[0]
        #find price
        print(disposition)
        price = apartment.find('p', class_='MuiTypography-root MuiTypography-body1 css-1ndcg2e').text.replace('\xa0', '').replace('Kč/měsíc', '')
        print(price)
        #find location
        location = apartment.findAll('p', class_='MuiTypography-root MuiTypography-body1 css-13ztabn')[1].text
        print(location)
    
        objekt_apartment = {
            'URL' : url_full,
            'Price': price,
            'Dimensions': dimensions,
            'Disposition': disposition,
            'Location' : location,
        }
        list_apartments.append(objekt_apartment)

    
    button = soup.find('button', 'MuiButtonBase-root MuiButton-root MuiButton-outlined MuiButton-outlinedInherit MuiButton-sizeMedium MuiButton-outlinedSizeMedium MuiButton-colorInherit MuiButton-root MuiButton-outlined MuiButton-outlinedInherit MuiButton-sizeMedium MuiButton-outlinedSizeMedium MuiButton-colorInherit css-lp5ywq').text

    if button != 'Další stránka':
        print('Již zde není žádná další strana. Končím')
        break

    strana += 1

    print(f'Scrapuju stránku č. {strana}')

<li class="MuiGrid-root MuiGrid-item css-l1328q" id="region-tip-item-1-3782468172"><style data-emotion="css qtq1ex">.css-qtq1ex{-webkit-text-decoration:underline;text-decoration:underline;text-decoration-color:var(--Link-underlineColor);--Link-underlineColor:rgba(25, 118, 210, 0.4);scroll-margin-top:16px;cursor:pointer;-webkit-text-decoration:none;text-decoration:none;color:#626D82;width:100%;}.css-qtq1ex:hover{text-decoration-color:inherit;}.css-qtq1ex:focus-visible{outline:0;box-shadow:0 0 0.188rem 0.063rem #03a9f4;}.css-qtq1ex:hover{color:#232B3A;}.css-qtq1ex:focus-visible{box-shadow:none;outline:#03a9f4 auto 1px;}.css-qtq1ex:visited .MuiTypography-root{color:#5E2F81;}</style><style data-emotion="css 1c7vz4z">.css-1c7vz4z{margin:0;font:inherit;line-height:inherit;letter-spacing:inherit;color:#1976d2;-webkit-text-decoration:underline;text-decoration:underline;text-decoration-color:var(--Link-underlineColor);--Link-underlineColor:rgba(25, 118, 210, 0.4);scroll-margin-top:16px;cursor:p

In [16]:
list_apartments

[{'URL': 'https://www.sreality.cz//detail/pronajem/byt/1+kk/zlin-zlin-sadova/670573132',
  'Price': '13500',
  'Dimensions': '40',
  'Disposition': '1+kk',
  'Location': 'Sadová, Zlín'},
 {'URL': 'https://www.sreality.cz//detail/pronajem/byt/1+kk/zlin-malenovice-tecovska/1112744524',
  'Price': '8300',
  'Dimensions': '26',
  'Disposition': '1+kk',
  'Location': 'Tečovská, Zlín - Malenovice'},
 {'URL': 'https://www.sreality.cz//detail/pronajem/byt/1+kk/zlin-zlin-j-a-bati/1042633804',
  'Price': 'Cena na vyžádání',
  'Dimensions': '31',
  'Disposition': '1+kk',
  'Location': 'J. A. Bati, Zlín'},
 {'URL': 'https://www.sreality.cz//detail/pronajem/byt/2+kk/zlin-zlin-lorencova/1472258636',
  'Price': '18700',
  'Dimensions': '65',
  'Disposition': '2+kk',
  'Location': 'Lorencova, Zlín'},
 {'URL': 'https://www.sreality.cz//detail/pronajem/byt/2+1/zlin-zlin-dlouha/1913504332',
  'Price': '12000',
  'Dimensions': '56',
  'Disposition': '2+1',
  'Location': 'Dlouhá, Zlín'},
 {'URL': 'https://

In [ ]:
list_apartments

In [ ]:
# URL
url = soup.find('a', class_='MuiTypography-root MuiTypography-inherit MuiLink-root MuiLink-underlineAlways css-1c7vz4z', attrs={'href'})['href']
print(url)
url_full = 'https://www.sreality.cz/' + url
url_full

In [ ]:
#dimenstions

dimensions = soup.find('p', class_='MuiTypography-root MuiTypography-body1 css-13ztabn').text.replace('Pronájem bytu ', '').replace('\xa0m²', '').split(' ')[1]
print(dimensions)

In [ ]:
# disposition
disposition = soup.find('p', class_='MuiTypography-root MuiTypography-body1 css-13ztabn').text.replace('Pronájem bytu ', '').split(' ')[0]
print(disposition)

In [ ]:
# price 
price = soup.find('p', class_='MuiTypography-root MuiTypography-body1 css-1ndcg2e').text.replace('\xa0', '').replace('Kč/měsíc', '')
print(price)

In [ ]:
# Location
location = soup.findAll('p', class_='MuiTypography-root MuiTypography-body1 css-13ztabn')[1].text
print(location)

### Datová analýza

2. Datová analýza - ETL proces
- V první fázi prověďte processing dat tak, aby obsahovala příslušné datové typy, nenulové hodnoty atp. Ulice i město bude v samostatných sloupcích.
- Zprocesovaná a vyčištěná data vyexportujte do `.csv` souboru s názvem `{vašeJméno}_zdrojová_data.csv`

In [3]:
import pandas as pd

df = pd.DataFrame(list_apartments)
df.head()
df.to_excel('apartments.xlsx')

In [4]:
df.head()

,URL,Price,Dimensions,Disposition,Location
0,https://www.sreality.cz//detail/pronajem/byt/2...,13500,45,2+1,"Na Honech II, Zlín"
1,https://www.sreality.cz//detail/pronajem/byt/1...,8300,26,1+kk,"Tečovská, Zlín - Malenovice"
2,https://www.sreality.cz//detail/pronajem/byt/1...,Cena na vyžádání,31,1+kk,"J. A. Bati, Zlín"
3,https://www.sreality.cz//detail/pronajem/byt/2...,18700,65,2+kk,"Lorencova, Zlín"
4,https://www.sreality.cz//detail/pronajem/byt/2...,12000,56,2+1,"Dlouhá, Zlín"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   URL          153 non-null    object
 1   Price        153 non-null    object
 2   Dimensions   153 non-null    object
 3   Disposition  153 non-null    object
 4   Location     153 non-null    object
dtypes: object(5)
memory usage: 6.1+ KB


In [10]:
# change Price to int and remove non-numeric values
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df = df.dropna(subset=['Price'])
df['Price'] = df['Price'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149 entries, 0 to 152
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   URL          149 non-null    object
 1   Price        149 non-null    int64 
 2   Dimensions   149 non-null    object
 3   Disposition  149 non-null    object
 4   Location     149 non-null    object
 5   City         149 non-null    object
 6   Street       136 non-null    object
dtypes: int64(1), object(6)
memory usage: 9.3+ KB


In [11]:
# change Dimensions to int and remove non-numeric values
df['Dimensions'] = pd.to_numeric(df['Dimensions'], errors='coerce')
df = df.dropna(subset=['Dimensions'])
df['Dimensions'] = df['Dimensions'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148 entries, 0 to 152
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   URL          148 non-null    object
 1   Price        148 non-null    int64 
 2   Dimensions   148 non-null    int64 
 3   Disposition  148 non-null    object
 4   Location     148 non-null    object
 5   City         148 non-null    object
 6   Street       135 non-null    object
dtypes: int64(2), object(5)
memory usage: 9.2+ KB


In [40]:
# split city and street 
df[['Street', 'City']] = df['Location'].str.split(', ', expand=True)
df

,Unnamed: 0,URL,Price,Dimensions,Disposition,Location,City,Street
0,0,https://www.sreality.cz//detail/pronajem/byt/2...,13500,45,2+1,"Na Honech II, Zlín",Zlín,Na Honech II
1,1,https://www.sreality.cz//detail/pronajem/byt/1...,8300,26,1+kk,"Tečovská, Zlín - Malenovice",Zlín - Malenovice,Tečovská
2,3,https://www.sreality.cz//detail/pronajem/byt/2...,18700,65,2+kk,"Lorencova, Zlín",Zlín,Lorencova
3,4,https://www.sreality.cz//detail/pronajem/byt/2...,12000,56,2+1,"Dlouhá, Zlín",Zlín,Dlouhá
4,5,https://www.sreality.cz//detail/pronajem/byt/1...,9000,25,1+1,"Vysoká, Zlín",Zlín,Vysoká
...,...,...,...,...,...,...,...,...
143,148,https://www.sreality.cz//detail/pronajem/byt/2...,18000,49,2+kk,"Tyršovo nábřeží, Zlín",Zlín,Tyršovo nábřeží
144,149,https://www.sreality.cz//detail/pronajem/byt/4...,19000,79,4+1,"Na Honech II, Zlín",Zlín,Na Honech II
145,150,https://www.sreality.cz//detail/pronajem/byt/4...,39900,100,4+kk,Luhačovice,None,Luhačovice
146,151,https://www.sreality.cz//detail/pronajem/byt/2...,9000,40,2+kk,"V Dolině, Zlín - Jaroslavice",Zlín - Jaroslavice,V Dolině


In [41]:
# Zprocesovaná a vyčištěná data vyexportujte do `.csv` souboru s názvem `{vašeJméno}_zdrojová_data.csv`
df.to_csv('EvaVonk_zdrojova_data.csv')


3. Datová analýza - analýza dat
- Třetí část začněte tak, že si do dataframe nahrajete vámi vyexportovaný `.csv` soubor s vyscrapovanými daty.
- Ve druhé fázi proveďte analýzu dat a odpovězte na tyto dotazy:
- `Jaká je průměrná cena bytů?`
- `Jaká je průměrná cena bytů pro každou kompozici (1+1, 2+1, atp.)? Zobrazte i na grafu.`
- `Jaká je průměrná velikost bytu pro každou kompozici?`
- `Existuje ulice, kde je vyšší koncentrace dražších bytů?`
- `Jaký typ kompozice je v daném městě nejčastěji inzerovan? Který to je? Proč myslíte, že tomu tak je?`
- `Existují zde inzerce bytů, které stojí více, než 20.000,-? Pokud ano, jsou v této cenové hladině inzerovány i maximálně dvoupokojové byty (2+1/2+kk)?`
- `Zjištěte minimální a maximální inzerce pro každou kompozici bytu. Která kompozice má největší rozptyl mezi minimální a maximální inzerovanou cenou?`

In [42]:
#Třetí část začněte tak, že si do dataframe nahrajete vámi vyexportovaný `.csv` soubor s vyscrapovanými daty.
df = pd.read_csv('EvaVonk_zdrojova_data.csv', delimiter=',')

In [70]:
#Jaká je průměrná cena bytů?
average_price = round(df['Price'].mean(), 0)
average_price
print(f'Average price of rental apartments in Zlin is: {average_price} CZK.')


Average price of rental apartments in Zlin is: 14069.0 CZK.


In [71]:
#Jaká je průměrná cena bytů pro každou kompozici (1+1, 2+1, atp.)? Zobrazte i na grafu.Celkovy pocet PCR testu
average_price_by_disposition = round(df.groupby('Disposition')['Price'].mean().reset_index(name= 'Average price by disposition'), 0)
print(average_price_by_disposition)

  Disposition  Average price by disposition
0         1+1                       10229.0
1        1+kk                       11377.0
2         2+1                       13347.0
3        2+kk                       16489.0
4         3+1                       16592.0
5        3+kk                       19125.0
6         4+1                       20000.0
7        4+kk                       39900.0
8    atypický                        9750.0
9      pokoje                        6890.0


In [72]:
#Jaká je průměrná velikost bytu pro každou kompozici?
average_size_by_disposition = round(df.groupby('Disposition')['Dimensions'].mean().reset_index(name= 'Average size by disposition'), 0)
print(average_size_by_disposition)

  Disposition  Average size by disposition
0         1+1                         35.0
1        1+kk                         30.0
2         2+1                         57.0
3        2+kk                         59.0
4         3+1                         78.0
5        3+kk                         83.0
6         4+1                         81.0
7        4+kk                        100.0
8    atypický                         27.0
9      pokoje                         38.0


In [73]:
#Existuje ulice, kde je vyšší koncentrace dražších bytů?
average_prices_by_streets = df.groupby('Street')['Price'].mean().reset_index()
average_prices_by_streets
highest_avg_prices = average_prices_by_streets.sort_values(by='Price', ascending=False).head(5)
print(highest_avg_prices)

         Street    Price
30  Na Honech I  25000.0
23    Lešetín I  20500.0
0     Bartošova  20000.0
65      Zabrání  20000.0
48      Přístav  20000.0


In [74]:
#Jaký typ kompozice je v daném městě nejčastěji inzerovan? Který to je? Proč myslíte, že tomu tak je?
most_common_disposition = df.groupby('Disposition').size().reset_index(name='Count')
most_common_disposition = most_common_disposition.sort_values(by='Count', ascending=False)
most_common_disposition
most_common = most_common_disposition.iloc[0]  # Nejčastější dispozice
print(f'Most common disposition is: {most_common['Disposition']} with {most_common['Count']} offers.')

Most common disposition is: 2+kk with 39 offers.


In [75]:
#Existují zde inzerce bytů, které stojí více, než 20.000,-? Pokud ano, jsou v této cenové hladině inzerovány i maximálně dvoupokojové byty (2+1/2+kk)?
over_20000 = df[df['Price'] > 20000]
over_20000_grouped = over_20000.groupby('Disposition').size().reset_index(name='Count')
over_20000_grouped = over_20000_grouped.sort_values(by='Count', ascending=False)
print(over_20000_grouped)

  Disposition  Count
0        2+kk      5
1        3+kk      2
2         4+1      1
3        4+kk      1


In [76]:
#Zjištěte minimální a maximální inzerce pro každou kompozici bytu. Která kompozice má největší rozptyl mezi minimální a maximální inzerovanou cenou?
stats = df.groupby('Disposition')['Price'].agg(['min', 'max']).reset_index()
print(stats)

  Disposition    min    max
0         1+1   4200  13500
1        1+kk   6500  14900
2         2+1   9000  16500
3        2+kk   9000  25000
4         3+1  11900  19000
5        3+kk  13000  29000
6         4+1  19000  21000
7        4+kk  39900  39900
8    atypický   7500  12000
9      pokoje   3708  10500
